## Exercises

All of the exercises for this module should be done within your ds-methodologies repository, inside of a directory named time_series.

The end result of this exercise should be a file named acquire.py.

1. Using the code from the lesson as a guide, create a dataframe named items that has all of the data for items.

2. Do the same thing, but for stores.

3. Extract the data for sales. There are a lot of pages of data here, so your code will need to be a little more complex. Your code should continue fetching data from the next page until all of the data is extracted.

4. Save the data in your files to local csv files so that it will be faster to access in the future.

5. Combine the data from your three separate dataframes into one large dataframe.

6. Acquire the Open Power Systems Data for Germany, which has been rapidly expanding its renewable energy production in recent years. The data set includes country-wide totals of electricity consumption, wind power production, and solar power production for 2006-2017. You can get the data here: https://raw.githubusercontent.com/jenfly/opsd/master/opsd_germany_daily.csv

7. Make sure all the work that you have done above is reproducible. That is, you should put the code above into separate functions in the acquire.py file and be able to re-run the functions and get the same data.

In [37]:
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

import requests

In [38]:
base_url = 'https://python.zach.lol'
print(requests.get(base_url).text)

{"api":"/api/v1","help":"/documentation"}



In [39]:
response = requests.get(base_url + '/documentation')
print(response.json()['payload'])


The API accepts GET requests for all endpoints, where endpoints are prefixed
with

    /api/{version}

Where version is "v1"

Valid endpoints:

- /stores[/{store_id}]
- /items[/{item_id}]
- /sales[/{sale_id}]

All endpoints accept a `page` parameter that can be used to navigate through
the results.



## Getting 'Items' data and creating a pandas dataframe 'items'

In [40]:
response = requests.get('https://python.zach.lol/api/v1/items')

data = response.json()
data.keys()

dict_keys(['payload', 'status'])

In [41]:
import pandas as pd

items = pd.DataFrame(data['payload']['items'])
items.head()

,item_brand,item_id,item_name,item_price,item_upc12,item_upc14
0,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013
1,Caress,2,Caress Velvet Bliss Ultra Silkening Beauty Bar...,6.44,11111065925,11111065925
2,Earths Best,3,Earths Best Organic Fruit Yogurt Smoothie Mixe...,2.43,23923330139,23923330139
3,Boars Head,4,Boars Head Sliced White American Cheese - 120 Ct,3.14,208528800007,208528800007
4,Back To Nature,5,Back To Nature Gluten Free White Cheddar Rice ...,2.61,759283100036,759283100036


In [42]:
response = requests.get(base_url + data['payload']['next_page'])
data = response.json()

print('max_page: %s' % data['payload']['max_page'])
print('next_page: %s' % data['payload']['next_page'])

items = pd.concat([items, pd.DataFrame(data['payload']['items'])]).reset_index()

max_page: 3
next_page: /api/v1/items?page=3


In [43]:
response = requests.get(base_url + data['payload']['next_page'])
data = response.json()

print('max_page: %s' % data['payload']['max_page'])
print('next_page: %s' % data['payload']['next_page'])

items = pd.concat([items, pd.DataFrame(data['payload']['items'])]).reset_index()

max_page: 3
next_page: None


In [44]:
items.head()

,level_0,index,item_brand,item_id,item_name,item_price,item_upc12,item_upc14
0,0,0.0,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013
1,1,1.0,Caress,2,Caress Velvet Bliss Ultra Silkening Beauty Bar...,6.44,11111065925,11111065925
2,2,2.0,Earths Best,3,Earths Best Organic Fruit Yogurt Smoothie Mixe...,2.43,23923330139,23923330139
3,3,3.0,Boars Head,4,Boars Head Sliced White American Cheese - 120 Ct,3.14,208528800007,208528800007
4,4,4.0,Back To Nature,5,Back To Nature Gluten Free White Cheddar Rice ...,2.61,759283100036,759283100036


## Getting 'stores' data and creating a pandas dataframe called 'stores'

In [45]:
response = requests.get('https://python.zach.lol/api/v1/stores')

data = response.json()
data.keys()

dict_keys(['payload', 'status'])

In [46]:
stores = pd.DataFrame(data['payload']['stores'])
stores.head()

,store_address,store_city,store_id,store_state,store_zipcode
0,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253
1,9255 FM 471 West,San Antonio,2,TX,78251
2,2118 Fredericksburg Rdj,San Antonio,3,TX,78201
3,516 S Flores St,San Antonio,4,TX,78204
4,1520 Austin Hwy,San Antonio,5,TX,78218


In [47]:
stores = pd.DataFrame(data['payload']['stores'])
stores.head()

,store_address,store_city,store_id,store_state,store_zipcode
0,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253
1,9255 FM 471 West,San Antonio,2,TX,78251
2,2118 Fredericksburg Rdj,San Antonio,3,TX,78201
3,516 S Flores St,San Antonio,4,TX,78204
4,1520 Austin Hwy,San Antonio,5,TX,78218


## Getting 'sales' data and creating a pandas dataframe called 'sales'

In [48]:
response = requests.get('https://python.zach.lol/api/v1/sales')

data = response.json()
data.keys()

dict_keys(['payload', 'status'])

In [49]:
data['payload']['max_page']

183

In [50]:
for i in range(1, data['payload']['max_page']):
    response = requests.get(base_url + data['payload']['next_page'])
    data = response.json()
    sales = pd.concat([sales, pd.DataFrame(data['payload']['sales'])])
sales = sales.reset_index()

In [52]:
sales.dtypes

index            int64
item             int64
sale_amount    float64
sale_date       object
sale_id          int64
store            int64
dtype: object

In [53]:
sales.head()

,index,item,sale_amount,sale_date,sale_id,store
0,0,50,77.0,"Wed, 15 Oct 2014 00:00:00 GMT",910001,9
1,1,50,52.0,"Thu, 16 Oct 2014 00:00:00 GMT",910002,9
2,2,50,65.0,"Fri, 17 Oct 2014 00:00:00 GMT",910003,9
3,3,50,66.0,"Sat, 18 Oct 2014 00:00:00 GMT",910004,9
4,4,50,81.0,"Sun, 19 Oct 2014 00:00:00 GMT",910005,9


In [54]:
sales.shape

(911000, 6)

### Write sales data to a csv

In [58]:
sales.to_csv(r'sales_data.csv')

### Acquire Open Power Systems Data for Germany

In [ ]:
# example code for reading the clipboard

# df = pd.read_clipboard(sep=',', 
#                        index_col=0, 
#                        names=['firstName', 
#                               'Name',
#                               'PClass',
#                               'Age',
#                               'Sex',
#                               'Survived',
#                               'SexCode'])

In [62]:
german_power_systems = pd.read_clipboard(sep=',')